In [2]:
# DATA INGESTION
from langchain_community.document_loaders import TextLoader
loader=TextLoader("1.txt")
text_document=loader.load()
text_document

[Document(metadata={'source': '1.txt'}, page_content='"HI MY NAME IS SIDDHANT"')]

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()


False

In [4]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

## load, chunk the content of webpage

loader=WebBaseLoader(web_paths=("https://en.wikipedia.org/wiki/Disease",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                     class_="mw-body")
                     ),)

text_document=loader.load()


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('medical.pdf')
Book_content=loader.load()

In [6]:
### LOAD IS DONE

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
book_chunk=text_splitter.split_documents(Book_content)


In [8]:
## VECTOR EMBEDDINGS in CHROMADB

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

embeddings_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)

db = Chroma.from_documents(book_chunk, embeddings)


In [ ]:
query="what are Unicellular Glands"
result=db.similarity_search(query)
result[0].page_content

'pharynx to be swallowed. Bacteria are then destroyed\nby the hydrochloric acid in the stomach. The air that\nreaches the lungs is almost entirely free of pathogens\nand particulate pollution.\nAnother location of ciliated epithelium in women is\nthe lining of the fallopian tubes. The cilia here sweep\nthe ovum, which has no means of self-locomotion,\ntoward the uterus.\nThe epithelial tissues are summarized in T able 4–1.\nGLANDS\nGlands are cells or organs that secrete something;\nthat is, they produce a substance that has a function\neither at that site or at a more distant site.\nUnicellular Glands\nUnicellular means “one cell.” Goblet cells are an\nexample of unicellular glands. As mentioned earlier,\ngoblet cells are found in the lining of the respiratory\nand digestive tracts. Their secretion is mucus (see also\nBox 4–1: Cystic Fibrosis).\nMulticellular Glands\nMost glands are made of many similar cells, or of a\nvariety of cells with their secretions mingled into a'

In [ ]:
## VECTOR EMBEDDINGS IN FAISSDB

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embeddings_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)

db1=FAISS.from_documents(book_chunk,embeddings)


In [ ]:
query="what are Unicellular Glands"
result1=db1.similarity_search(query)
result1[0].page_content

/Volumes/MySSD/Siddhant Gupta/projects/LANGCHAIN/Chatbot/venv/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


'pharynx to be swallowed. Bacteria are then destroyed\nby the hydrochloric acid in the stomach. The air that\nreaches the lungs is almost entirely free of pathogens\nand particulate pollution.\nAnother location of ciliated epithelium in women is\nthe lining of the fallopian tubes. The cilia here sweep\nthe ovum, which has no means of self-locomotion,\ntoward the uterus.\nThe epithelial tissues are summarized in T able 4–1.\nGLANDS\nGlands are cells or organs that secrete something;\nthat is, they produce a substance that has a function\neither at that site or at a more distant site.\nUnicellular Glands\nUnicellular means “one cell.” Goblet cells are an\nexample of unicellular glands. As mentioned earlier,\ngoblet cells are found in the lining of the respiratory\nand digestive tracts. Their secretion is mucus (see also\nBox 4–1: Cystic Fibrosis).\nMulticellular Glands\nMost glands are made of many similar cells, or of a\nvariety of cells with their secretions mingled into a'

In [ ]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain.chains.combine_documents import create_stuff_documents_chain
llm = ChatGroq(model="llama3-8b-8192")
load_dotenv()
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template('''
Answer the following based only on the provided context.
Think step by step before answering.
I will tip you $1000 if answers are correct.
<context>{context}</context>
Question: {input}
''')
document_chain=create_stuff_documents_chain(llm,prompt)

In [ ]:
retreiver=db1.as_retriever()
retreiver

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x313057d90>, search_kwargs={})

In [ ]:
from langchain.chains import create_retrieval_chain
retreival_chain=create_retrieval_chain(retreiver,document_chain)

In [ ]:
retreival_chain.invoke({"input":"What is Epithelial Tissue"})

/Volumes/MySSD/Siddhant Gupta/projects/LANGCHAIN/Chatbot/venv/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'input': 'What is Epithelial Tissue',
 'context': [Document(id='933b238d-cedb-4f9d-a8bf-0a6999e3cb86', metadata={'producer': 'Acrobat Distiller 7.0.5 (Windows)', 'creator': 'Adobe Acrobat 7.0', 'creationdate': '2007-01-12T08:57:45-05:00', 'author': 'Scanlon, Valerie C.; Sanders, Tina', 'moddate': '2007-04-30T22:33:49+02:00', 'title': 'Essentials of Anatomy and Physiology', 'source': 'medical.pdf', 'total_pages': 622, 'page': 21, 'page_label': '22'}, page_content='chemical reactions. Cell structure and function are\ndiscussed in Chapter 3.\nTISSUES\nA tissue is a group of cells with similar structure and\nfunction. There are four groups of tissues:\nEpithelial tissues—cover or line body surfaces; some\nare capable of producing secretions with speciﬁc\nfunctions. The outer layer of the skin and sweat\nglands are examples of epithelial tissues. Internal\nepithelial tissues include the walls of capillaries\n(squamous epithelium) and the kidney tubules\n(cuboidal epithelium), as shown in F

In [ ]:
response=retreival_chain.invoke({"input":"who is the author of this book"})
response['answer']

/Volumes/MySSD/Siddhant Gupta/projects/LANGCHAIN/Chatbot/venv/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


'A simple one!\n\nAccording to the context, the authors of this book are:\n\n1. Valerie C. Scanlon\n2. Tina Sanders'